# Notebook 

## Guía paso a paso del ETL (`etl_clv.ipynb`)

| # | Celda | ¿Qué hacemos aquí? | ¿Por qué es importante? |
|---|-------|--------------------|-------------------------|
| 0 | **Imports & versión**<br>`from pathlib import Path`<br>`import pandas as pd` | Cargamos las librerías base y mostramos la versión de Pandas (`2.3.x`). | Garantiza que el entorno es el esperado; cualquier discrepancia de versión se detecta al instante. |
| 1 | **Carga del CSV**<br>`pd.read_csv(...)` | Leemos `online_retail_II.csv`, definimos tipos (`dtype`) y parseamos fechas (`parse_dates`). | Traemos los datos brutos a memoria en un *DataFrame* coherente, listo para inspección. |
| 2 | **Inspección rápida**<br>`df.info()` + `isna()` | Vemos tipos, nº de filas, nulos y estadísticas básicas. | Diagnóstico inicial: detectamos columnas problemáticas y validamos que la carga fue correcta. |
| 3 | **Filtrado de devoluciones y precios inválidos**<br>`Quantity > 0`, `Price > 0` | Eliminamos devoluciones (`Quantity < 0`) y errores de precio. | Evitamos que operaciones negativas o precios cero distorsionen métricas de ventas y modelos CLV. |
| 4 | **Limpieza y enriquecimiento**<br>`InvoiceDate → datetime` + `Sales = Quantity * Price` | Convertimos la fecha a formato nativo y creamos la columna `Sales` (ingreso por línea). | Tipos correctos = operaciones vectorizadas rápidas; `Sales` es la métrica monetaria clave para CLV. |
| 5 | **Reordenar columnas (opcional)** | Ordenamos columnas para legibilidad (`Invoice`, `InvoiceDate`, …). | Facilita lectura humana y consistencia con otros artefactos (Parquet, dashboards). |
| 6 | **Persistencia en Parquet**<br>`to_parquet(..., compression='snappy')` | Guardamos el *DataFrame* limpio en `data/processed/clv.parquet`. | Parquet es compacto y se lee 5-10× más rápido que CSV; `snappy` equilibra compresión y velocidad. |
| 7 | **Validación de lectura**<br>`pd.read_parquet(...)` | Volvemos a cargar el Parquet y mostramos `info()` + `head()`. | Prueba de fuego: confirma que el archivo es legible y mantiene los tipos esperados. |
| 8 | **Mini-cohortes (opcional)** | Agregamos por mes (`dt.to_period('M')`) y calculamos `n_customers` y `revenue`. | Primer insight de negocio: tamaño de cohorte y facturación mensual sirven para sanity-check y gráficos iniciales. |


**Resultado de la celda 6:** `data/processed/clv.parquet` (~40-50 MB) es la base única de verdad para todos los modelos y visualizaciones posteriores.

## 0. Importación y versión

In [7]:
from pathlib import Path
import pandas as pd

print("Pandas:", pd.__version__)

Pandas: 2.2.3


## 1. Carga del DataSet


In [12]:
DATA_FILE = Path("../data/raw/online_retail_II.csv")   # ajusta nombre si difiere

df = pd.read_csv(
    DATA_FILE,
    dtype={
        "Customer ID": "Int64",    # enteros con soporte de nulos
        "Invoice": "string"
    },
    parse_dates=["InvoiceDate"],
    encoding="ISO-8859-1",         # evita problemas con acentos
)

df.shape, df.head()

((1067371, 8),
   Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
           InvoiceDate  Price  Customer ID         Country  
 0 2009-12-01 07:45:00   6.95        13085  United Kingdom  
 1 2009-12-01 07:45:00   6.75        13085  United Kingdom  
 2 2009-12-01 07:45:00   6.75        13085  United Kingdom  
 3 2009-12-01 07:45:00   2.10        13085  United Kingdom  
 4 2009-12-01 07:45:00   1.25        13085  United Kingdom  )

## 2. Inspección rápida



In [13]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  string        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   Int64         
 7   Country      1067371 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(3), string(1)
memory usage: 66.2+ MB


Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

## 3. Filtrar devoluciones

In [15]:
df = df.loc[df["Quantity"] > 0]
df = df.loc[df["Price"] > 0]
df = df.dropna(subset=["Customer ID"]).copy()

## 4. Columna Sales

In [16]:
df["Sales"] = df["Quantity"] * df["Price"]
df.rename(columns=lambda c: c.strip(), inplace=True)

## 5. Reordenar columnas

In [18]:
cols = ["Invoice", "InvoiceDate", "Customer ID", "Country",
        "StockCode", "Description", "Quantity", "Price", "Sales"]
df = df[cols]
df.head()

,Invoice,InvoiceDate,Customer ID,Country,StockCode,Description,Quantity,Price,Sales
0,489434,2009-12-01 07:45:00,13085,United Kingdom,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,83.4
1,489434,2009-12-01 07:45:00,13085,United Kingdom,79323P,PINK CHERRY LIGHTS,12,6.75,81.0
2,489434,2009-12-01 07:45:00,13085,United Kingdom,79323W,WHITE CHERRY LIGHTS,12,6.75,81.0
3,489434,2009-12-01 07:45:00,13085,United Kingdom,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,100.8
4,489434,2009-12-01 07:45:00,13085,United Kingdom,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,30.0


## 6. Persistencia

In [19]:
OUTPUT_DIR = Path("../data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = OUTPUT_DIR / "clv.parquet"
df.to_parquet(OUTPUT_FILE, compression="snappy")

print("File written:", OUTPUT_FILE, "→",
      round(OUTPUT_FILE.stat().st_size / 1e6, 2), "MB")

File written: ../data/processed/clv.parquet → 10.08 MB


## 7. Validación de lectura

In [20]:
clean = pd.read_parquet(OUTPUT_FILE)
clean.info()
clean.head()

<class 'pandas.core.frame.DataFrame'>
Index: 805549 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805549 non-null  string        
 1   InvoiceDate  805549 non-null  datetime64[ns]
 2   Customer ID  805549 non-null  Int64         
 3   Country      805549 non-null  object        
 4   StockCode    805549 non-null  object        
 5   Description  805549 non-null  object        
 6   Quantity     805549 non-null  int64         
 7   Price        805549 non-null  float64       
 8   Sales        805549 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(1), object(3), string(1)
memory usage: 62.2+ MB


,Invoice,InvoiceDate,Customer ID,Country,StockCode,Description,Quantity,Price,Sales
0,489434,2009-12-01 07:45:00,13085,United Kingdom,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,83.4
1,489434,2009-12-01 07:45:00,13085,United Kingdom,79323P,PINK CHERRY LIGHTS,12,6.75,81.0
2,489434,2009-12-01 07:45:00,13085,United Kingdom,79323W,WHITE CHERRY LIGHTS,12,6.75,81.0
3,489434,2009-12-01 07:45:00,13085,United Kingdom,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,100.8
4,489434,2009-12-01 07:45:00,13085,United Kingdom,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,30.0


## 8. Mini-cohortes

### ¿Qué son los cohortes y por qué importan?

Un **cohorte** es un grupo de clientes que comparte una misma “fecha de nacimiento” según un criterio definido—por ejemplo, el mes de **su primera compra**.  
Al separar los usuarios por cohortes podemos analizar cómo evoluciona cada “generación” sin mezclarla con las demás.

| Tipo de cohorte | Criterio de agrupación | Ejemplo |
|-----------------|------------------------|---------|
| **Adquisición** | Mes/Semana/Día de la primera compra | Clientes captados en **Ene-2020** |
| Activación | Fecha de primera acción clave (instalar app, registrarse) | Usuarios que activaron el plan **Q1-2024** |
| Campaña | Fuente o canal de entrada | Leads originados por **Google Ads** |

### ¿Por qué lo necesitamos para CLV?

1. **Medir retención** – ¿Sigue comprando el 40 % de la cohorte después de 6 meses?  
2. **Detectar anomalías** – Un pico o un desplome en una cohorte concreta revela un error de carga o un cambio de mercado.  
3. **Variables del modelo** – La “edad del cliente” (tiempo desde su cohorte) alimenta modelos de supervivencia como **Pareto/NBD**.  
4. **Segmentación de negocio** – Marketing compara la rentabilidad de campañas o temporadas sin mezclar generaciones de clientes.


In [21]:
cohort = (
    clean
    .groupby(clean["InvoiceDate"].dt.to_period("M"))
    .agg(n_customers=("Customer ID", "nunique"),
         revenue=("Sales", "sum"))
    .reset_index()
)
cohort.head()

,InvoiceDate,n_customers,revenue
0,2009-12,955,686654.160
1,2010-01,720,557319.062
2,2010-02,772,506371.066
3,2010-03,1057,699608.991
4,2010-04,942,594609.192
